In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import random
import os

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [ ]:
def my_seed_everywhere(seed: int = 42):
    random.seed(seed) # random
    np.random.seed(seed) # np
    os.environ["PYTHONHASHSEED"] = str(seed) # os
    tf.random.set_seed(seed) # tensorflow

my_seed = 42
my_seed_everywhere(my_seed)

In [ ]:
Mytrain = train[train.columns[:-2]]

In [ ]:
train.groupby(['subject'])['Activity'].value_counts()

subject  Activity          
1        WALKING               95
         STANDING              53
         WALKING_UPSTAIRS      53
         LAYING                50
         WALKING_DOWNSTAIRS    49
                               ..
30       WALKING               65
         WALKING_UPSTAIRS      65
         SITTING               62
         WALKING_DOWNSTAIRS    62
         STANDING              59
Name: count, Length: 126, dtype: int64

In [ ]:
len(train['subject'].unique())

21

In [ ]:
train['Activity'].value_counts()

Activity
LAYING                1407
STANDING              1374
SITTING               1286
WALKING               1226
WALKING_UPSTAIRS      1073
WALKING_DOWNSTAIRS     986
Name: count, dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
Mytrain_scaled = scaler.fit_transform(Mytrain)

In [ ]:
y_train = train['Activity']

activity_mapping = {
    'WALKING': 0,
    'WALKING_UPSTAIRS': 1,
    'WALKING_DOWNSTAIRS': 2,
    'SITTING': 3,
    'STANDING': 4,
    'LAYING': 5
}


y_train = [activity_mapping[activity] for activity in y_train]

In [ ]:
def build_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(256, activation='relu'),
      tf.keras.layers.Dense(6, activation='softmax')
  ])

  model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
  return model

In [ ]:
k = 10
num_val = len(Mytrain) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)

    X_val = Mytrain[i * num_val: (i + 1) * num_val]
    y_val = y_train[i * num_val: (i + 1) * num_val]

    X_train_part = np.concatenate([Mytrain[:i * num_val], Mytrain[(i + 1) * num_val:]], axis=0)
    y_train_part = np.concatenate([y_train[:i * num_val], y_train[(i + 1) * num_val:]], axis=0)

    model = build_model()
    model.fit(X_train_part, y_train_part, epochs=num_epochs, verbose=1)

    val_loss, val_acc = model.evaluate(X_val, np.array(y_val), verbose=0)
    all_scores.append(val_acc)


processing fold # 0
Epoch 1/100
207/207 [==============================] - 2s 3ms/step - loss: 0.3652 - accuracy: 0.8380
Epoch 2/100
207/207 [==============================] - 1s 3ms/step - loss: 0.1544 - accuracy: 0.9359
Epoch 3/100
207/207 [==============================] - 1s 3ms/step - loss: 0.1097 - accuracy: 0.9553
Epoch 4/100
207/207 [==============================] - 1s 3ms/step - loss: 0.0938 - accuracy: 0.9622
Epoch 5/100
207/207 [==============================] - 1s 3ms/step - loss: 0.1088 - accuracy: 0.9613
Epoch 6/100
207/207 [==============================] - 1s 3ms/step - loss: 0.0802 - accuracy: 0.9692
Epoch 7/100
207/207 [==============================] - 1s 3ms/step - loss: 0.0772 - accuracy: 0.9678
Epoch 8/100
207/207 [==============================] - 1s 3ms/step - loss: 0.0727 - accuracy: 0.9749
Epoch 9/100
207/207 [==============================] - 1s 3ms/step - loss: 0.0584 - accuracy: 0.9767
Epoch 10/100
207/207 [==============================] - 1s 3ms/step - l

In [ ]:
all_scores

[0.9646258354187012,
 0.878911554813385,
 0.9659863710403442,
 0.8612244725227356,
 0.8326530456542969,
 0.9197278618812561,
 0.9795918464660645,
 0.9360544085502625,
 0.921088457107544,
 0.9904761910438538]

In [ ]:
mean_score = np.mean(all_scores)
mean_score

0.9250340044498444

### 결과1

In [ ]:
all_scores

# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(256, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(256, activation='relu'),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(6, activation='softmax')
# ])

[0.9755101799964905,
 0.8761904835700989,
 0.9523809552192688,
 0.8544217944145203,
 0.8734694123268127,
 0.9387755393981934,
 0.9088435173034668,
 0.8925170302391052,
 0.9360544085502625,
 0.9374149441719055]

In [ ]:
np.mean(all_scores)

0.9145578265190124

### 결과2

In [ ]:
all_scores

#       tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')

[0.961904764175415,
 0.9034013748168945,
 0.9646258354187012,
 0.8721088171005249,
 0.8938775658607483,
 0.9142857193946838,
 0.9714285731315613,
 0.9292517304420471,
 0.956462562084198,
 0.9850339889526367]

In [ ]:
mean_score = np.mean(all_scores)
mean_score

0.9352380931377411

### 결과3

In [ ]:
all_scores

    # model = tf.keras.Sequential([
    #     tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
    #     tf.keras.layers.Dropout(0.5),
    #     tf.keras.layers.Dense(300, activation='relu'),
    #     tf.keras.layers.Dropout(0.5),
    #     tf.keras.layers.Dense(300, activation='relu'),
    #     tf.keras.layers.Dropout(0.5),
    #     tf.keras.layers.Dense(300, activation='relu'),
    #     tf.keras.layers.Dropout(0.5),
    #     tf.keras.layers.Dense(6, activation='softmax')
    # ])

[0.9768707752227783,
 0.8816326260566711,
 0.9632652997970581,
 0.8190476298332214,
 0.8653061389923096,
 0.9414966106414795,
 0.9387755393981934,
 0.801360547542572,
 0.8108843564987183,
 0.9537414908409119]

In [ ]:

mean_score = np.mean(all_scores)
mean_score

0.8952381014823914

### 결과 4

In [ ]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(250, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(250, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model

# all_scores
# 0.9727891087532043,
#  0.8680272102355957,
#  0.9401360750198364,
#  0.8557823300361633,
#  0.8802720904350281,
#  0.9278911352157593,
#  0.9768707752227783,
#  0.9428571462631226,
#  0.956462562084198,
#  0.9782313108444214]

# mean_score = np.mean(all_scores)
# 0.9299319744110107


### 결과 5

In [ ]:
# def build_model():
#   model = tf.keras.Sequential([
#       tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(300, activation='relu'),
#       tf.keras.layers.Dense(6, activation='softmax')
#   ])

#   model.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
#   return model
all_scores

[0.9591836929321289,
 0.9006802439689636,
 0.9401360750198364,
 0.8340135812759399,
 0.8680272102355957,
 0.9401360750198364,
 0.9632652997970581,
 0.9333333373069763,
 0.9455782175064087,
 0.9904761910438538]

In [ ]:
mean_score = np.mean(all_scores)
mean_score

0.9274829924106598

### 결과 6

In [ ]:
  # model = tf.keras.Sequential([
  #     tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
  #     tf.keras.layers.Dense(300, activation='relu'),
  #     tf.keras.layers.Dense(300, activation='relu'),
  #     tf.keras.layers.Dense(6, activation='softmax')
  # ])
all_scores

[0.9727891087532043,
 0.8734694123268127,
 0.9700680375099182,
 0.8761904835700989,
 0.8816326260566711,
 0.9319728016853333,
 0.9782313108444214,
 0.9061224460601807,
 0.956462562084198,
 0.9809523820877075]

In [ ]:
mean_score = np.mean(all_scores)
mean_score

0.9327891170978546

### 결과 7

In [ ]:
  # model = tf.keras.Sequential([
  #     tf.keras.layers.Dense(300, activation='relu', input_shape=(Mytrain_scaled.shape[1],)),
  #     tf.keras.layers.Dense(300, activation='relu'),
  #     tf.keras.layers.Dense(6, activation='softmax')
  # ])
all_scores

[0.9768707752227783,
 0.8734694123268127,
 0.9578231573104858,
 0.8326530456542969,
 0.8149659633636475,
 0.9238095283508301,
 0.9795918464660645,
 0.9319728016853333,
 0.9442176818847656,
 0.9850339889526367]

In [ ]:
mean_score = np.mean(all_scores)
mean_score

0.9220408201217651

### 최종

### 결과 2번이 0.9352380931377411 로 가장 높음